# 3D interpretation SSRM data

In [ ]:
import plotly as py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import LinearNDInterpolator
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from skimage.transform import resize

In [ ]:
data_path = '___' #data folder location
file_name = '____.xlsx' #SSRM raw dataset for interpolation

In [ ]:
data_path+file_name

In [ ]:
df = pd.read_excel(data_path+file_name)

In [ ]:
df.head() #see excel file

In [ ]:
inc = 1 # set increment between X and Y points, 1 for full adoption of all datapoints.
Xv = df["X"].values[::inc] #X value (nm) for scan size
Yv = df["Y"].values[::inc] #Y value (nm) for scan size
Z0 = df["Z0"].values[::inc] #Z value (nm) for first experimental scan
Z1 = df["Z1"].values[::inc] #Z value (nm) for second experimental scan
Z2 = df["Z2"].values[::inc] #Z value (nm) for third experimental scan
Z3 = df["Z3"].values[::inc] #Z value (nm) for fourth experimental scan
Re0 = df["R3"].values[::inc] #Resistivity value for first experimental scan
Re1 = df["R2"].values[::inc] #Resistivity value for second experimental scan
Re2 = df["R1"].values[::inc] #Resistivity value for third experimental scan
Re3 = df["R0"].values[::inc] #Resistivity value for fourth experimental scan

In [ ]:
#See 2-D data layout prior to interpolation
plt.figure(figsize=(20,20))
plt.scatter(Xv, Yv)
plt.xlim([0,127]) #for 128 x 128 data
plt.ylim([0,127]) #for 128 x 128 data
plt.grid()
plt.show()

In [ ]:
dx = 1 #interpolation x increment
dy = 1 #interpolation y increment
dz = 10 #interpolation z increment between experimental layers

In [ ]:
#define terms prior to interpolation
x_temp = Xv 
for i in range(3): 
    x = np.append(x_temp, Xv )
    x_temp = x
    
y_temp = Yv 
for i in range(3): 
    y = np.append(y_temp, Yv )
    y_temp = y
    
z_temp = Z0
z_extras = [Z1, Z2, Z3]
for z in z_extras: 
    z_data = np.append(z_temp, z)
    z_temp = z_data
    
Re_temp = Re0
Re_extras = [Re1, Re2, Re3]
for Re in Re_extras: 
    Re_data = np.append(Re_temp, Re)
    Re_temp = Re_data

In [ ]:
# Create coordinate pairs
cartcoord = list(zip(x, y, z_data))

# create meshgrid
X = np.arange(min(x), max(x)+dx, dx)
Y = np.arange(min(y), max(y)+dy, dy)
Z = np.arange(min(z_data), max(z_data)+dz, dz)
X, Y, Z = np.meshgrid(X, Y, Z)

In [ ]:
# interpolate depth values
interp = LinearNDInterpolator(cartcoord, Re_data, fill_value=0)

In [ ]:
Re_new = interp(X, Y, Z)

In [ ]:
Re_new.shape #see size of interpolated dataset (x points, y points, z layers)

In [ ]:
Re_new[:,:,0].shape

In [ ]:
#See experimental layers with those obtained via interpolation
n_layers = Re_new.shape[2]
plt.figure(figsize=(15, 15))
for i in range(n_layers): 
    Re_layer = Re_new[:,:,i]
    plt.subplot(5,5,i+1)
    plt.pcolormesh(Re_layer)

plt.tight_layout()
#plt.colorbar()
plt.show()

In [ ]:
x_flat, y_flat, z_flat = X.flatten(), Y.flatten(), Z.flatten()
Re_flat = Re_new.flatten()

In [ ]:
use_cols = [0,4,6,8] #bring in experimental resistivity data for histogram generation
data_cols = df.iloc[:,use_cols].values

In [ ]:
vals = plt.hist(data_cols.reshape(-1,1),bins=60, range=[-9,3]) #produce histogram from raw resistivity data with range and #bins
counts = vals[0]
res    = vals[1]

In [ ]:
for c in counts: 
    print(c) #print histogram data with given range and bins, for importation to graphical software

In [ ]:
vals = plt.hist([Re0,Re1,Re2,Re3],bins=60, range=[-9,3], stacked=True) #view stacked histogram from separate experimental layers

In [ ]:
#Examine 3-D Volumes based on threshold values; color, threshold range, and aspect ratio readily adaptable to individual datasets
layout = go.Layout(
             scene=dict(
                 aspectmode='data'
         ))
fig = go.Figure(data=go.Volume(
    x=x_flat, y=y_flat, z=z_flat*0.35, #aspect ratio control
    value=Re_flat,
    opacity=0.45, #set opacity
    isomin=-2.5, #set resistivity range for thresholding volumes, isomin=lower threshold
    isomax=-1, #set resistivity range for thresholding volumes, isomax=upper threshold
    #colorscale='Plasma',#use plasma if not thresholding volumes
    #colorscale=[[0, 'rgb(127,8,165)'], [1, 'rgb(127,8,165)']], #Purple
    colorscale=[[0, 'rgb(251,166,55)'], [1, 'rgb(251,166,55)']], #Gold
    #colorscale=[[0, 'rgb(12,42,80)'], [1, 'rgb(12,42,80)']], #Blue
    #colorscale=[[0, 'rgb(246,143,70)'], [1, 'rgb(246,143,70)']], #Orange
    #colorscale=[[0, 'rgb(222,112,101)'], [1, 'rgb(222,112,101)']], #Pink
    caps=dict(x_show=True, y_show=True),
    surface_count=5,
    ),layout=layout)
fig.update_layout(scene_xaxis_showticklabels=False,
                  scene_yaxis_showticklabels=False,
                  scene_zaxis_showticklabels=False)
fig.show()

In [ ]:
#prepare layers for resistivity volume from experimental and interpolated layers
Re_slices = []
Re_slices_nan = []
Re_Slices_Ca = []

for i in range(Re_new.shape[2]): 
    Re_nan = Re_new[:,:,i]
    Re_nan[Re_nan == 0] = np.nan   
    Re_slices_nan.append(Re_nan)
    Re_slices.append(Re_new[:,:,i])

In [ ]:
#view layers for a resistivity volume from experimental and interpolated layers
OP=0.45 #set opacity
SEP=3 #Set z separation between layers
MIN=-9 #Set lower datascale range
MAX=-1 #Set upper datascale range
data = [
    go.Surface(z=Re_slices[0], surfacecolor=Re_slices[0], opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[1]+SEP, surfacecolor=Re_slices[1],  showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[2]+2*SEP, surfacecolor=Re_slices[2], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[3]+3*SEP, surfacecolor=Re_slices[3], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[4]+4*SEP, surfacecolor=Re_slices[4], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[5]+5*SEP, surfacecolor=Re_slices[5], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[6]+6*SEP, surfacecolor=Re_slices[6], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[7]+7*SEP, surfacecolor=Re_slices[7], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[8]+8*SEP, surfacecolor=Re_slices[8], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[9]+9*SEP, surfacecolor=Re_slices[9], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[10]+10*SEP, surfacecolor=Re_slices[10], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[11]+11*SEP, surfacecolor=Re_slices[11], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
    go.Surface(z=Re_slices[12]+12*SEP, surfacecolor=Re_slices[12], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX),
 #   go.Surface(z=Re_slices[13]+13*SEP, surfacecolor=Re_slices[13], showscale=False, opacity=OP, colorscale='Plasma', cauto=False, cmin=MIN, cmax=MAX)]
]
contours = {
        "z": {"show": True, "start": 0.5, "end": 0.8, "Rze": 0.05}
    },
layout2 = go.Layout(
             scene=dict(
                 aspectmode='manual',
                 aspectratio=go.layout.scene.Aspectratio(
        x=1, y=1, z=0.4) #set aspect ratio
             )) 
fig = go.Figure(data=data,layout=layout2)
fig.update_layout(scene_xaxis_showticklabels=False,
                  scene_yaxis_showticklabels=False,
                  scene_zaxis_showticklabels=False)
fig.show()